In [ ]:
#align_images_ECC
#再次修正版本使輸出不亂跑
#不切變與縮放，只有旋轉和平移避免讓原圖失真

import cv2
import numpy as np
import os

def align_images_ecc(reference_image, target_image):
    """
    使用 ECC 方法對齊影像，確保輸出維持 A4 大小。
    只允許平移和旋轉，避免切變和縮放。
    :param reference_image: 參考影像 (A4 大小)
    :param target_image: 目標影像 (A4 大小)
    :return: 對齊後的目標影像 (A4 大小)
    """
    # 確保輸入影像為 A4 大小
    a4_width, a4_height = 5100, 7021  # A4 尺寸 (600 DPI)
    
    # 調整輸入影像至 A4 大小
    reference_image = cv2.resize(reference_image, (a4_width, a4_height))
    target_image = cv2.resize(target_image, (a4_width, a4_height))

    # 轉換為灰階
    reference_gray = cv2.cvtColor(reference_image, cv2.COLOR_BGR2GRAY)
    target_gray = cv2.cvtColor(target_image, cv2.COLOR_BGR2GRAY)

    # 初始化變換矩陣
    warp_matrix = np.eye(2, 3, dtype=np.float32)  # 2x3 矩陣，適用於平移和旋轉

    # 設定停止條件
    criteria = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 5000, 1e-10)

    try:
        # 執行 ECC 影像註冊
        cc, warp_matrix = cv2.findTransformECC(
            reference_gray, target_gray, warp_matrix, cv2.MOTION_EUCLIDEAN, 
            criteria, None, 5
        )

        # 套用變換並確保輸出為 A4 大小
        aligned_image = cv2.warpAffine(
            target_image, warp_matrix,
            (a4_width, a4_height),
            flags=cv2.INTER_LINEAR + cv2.WARP_INVERSE_MAP,
            borderMode=cv2.BORDER_CONSTANT,
            borderValue=(0, 0, 0)  # 黑色填充
        )
    except cv2.error:
        print("警告：對齊失敗，返回原始大小的目標影像")
        aligned_image = target_image

    return aligned_image


def ensure_a4_size(image):
    """
    確保影像為A4大小，必要時進行填充或縮放。
    :param image: 輸入影像
    :return: A4大小的影像
    """
    a4_width, a4_height = 4960, 7014  # A4 尺寸 (600 DPI)
    
    # 創建A4畫布
    a4_canvas = np.full((a4_height, a4_width, 3), (0, 0, 0), dtype=np.uint8)
    
    if image is not None:
        # 計算縮放比例，保持原始比例
        h, w = image.shape[:2]
        scale_w = a4_width / w
        scale_h = a4_height / h
        scale = min(scale_w, scale_h)

        # 調整影像大小
        new_width = int(w * scale)
        new_height = int(h * scale)
        resized_image = cv2.resize(image, (new_width, new_height))

        # 計算居中位置
        y_offset = (a4_height - new_height) // 2
        x_offset = (a4_width - new_width) // 2

        # 將調整後的影像放置在A4畫布上
        a4_canvas[y_offset:y_offset + new_height, x_offset:x_offset + new_width] = resized_image

    return a4_canvas

def process_images(folder_path, output_folder):
    """
    主函數：處理所有影像並確保輸出為A4大小。
    """
    # 建立輸出資料夾
    os.makedirs(output_folder, exist_ok=True)

    # 讀取所有影像檔案
    image_files = sorted([f for f in os.listdir(folder_path) 
                         if f.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp'))])
    
    if not image_files:
        raise ValueError("資料夾中未找到影像檔案")

    # 讀取並處理參考影像
    reference_path = os.path.join(folder_path, image_files[0])
    reference_image = cv2.imread(reference_path)
    if reference_image is None:
        raise ValueError(f"無法讀取參考影像: {reference_path}")

    # 確保參考影像為A4大小
    reference_image = ensure_a4_size(reference_image)
    
    # 儲存處理後的參考影像
    output_path = os.path.join(output_folder, f"aligned_a4_1.png")
    cv2.imwrite(output_path, reference_image)
    print(f"已儲存參考影像: {output_path}")

    # 處理其餘影像
    for i, image_file in enumerate(image_files[1:], 2):
        target_path = os.path.join(folder_path, image_file)
        target_image = cv2.imread(target_path)
        
        if target_image is None:
            print(f"警告：無法讀取影像 {target_path}，跳過處理")
            continue

        # 確保目標影像為A4大小
        target_image = ensure_a4_size(target_image)
        
        # 對齊影像
        aligned_image = align_images_ecc(reference_image, target_image)
        
        # 再次確保輸出為A4大小
        final_image = ensure_a4_size(aligned_image)
        
        # 儲存結果
        output_path = os.path.join(output_folder, f"aligned_a4_{i}.png")
        cv2.imwrite(output_path, final_image)
        print(f"已儲存對齊影像: {output_path}")

if __name__ == "__main__":
    input_folder = r"F:\Shitephen\Output log for Hitachi sugi from ARATA\Hitachi Sugi 9th take (1-305)\No2_kamitsuga7_16\20240702_162309\postprocess"
    output_folder = r"F:\Shitephen\Output log for Hitachi sugi from ARATA\Hitachi Sugi 9th take (1-305)\No2_kamitsuga7_16\20240702_162309\postprocess\aligned_cropped_images"
    process_images(input_folder, output_folder)

In [ ]:
# ECC (Enhanced Correlation Coefficient) fitting
# 目前最好
# 加上了區域連通性分析
# 篩噪點在計算生長和分解量之後
# 使平移+旋轉+切變+縮放 試圖對準各種形狀的根

import cv2
import os
import numpy as np
import pandas as pd

# 資料夾路徑
input_folder = r"F:\Shitephen\Output log for Hitachi sugi from ARATA\Hitachi Sugi 9th take (1-305)\No2_kamitsuga7_16\20240702_162309\postprocess\aligned_cropped_images"
output_csv = r"F:\Shitephen\Output log for Hitachi sugi from ARATA\Hitachi Sugi 9th take (1-305)\No2_kamitsuga7_16\20240702_162309\postprocess\results_ECC_CCA.csv"
output_visual_folder = os.path.join(input_folder, "visual_results_ECC_CCA")

os.makedirs(output_visual_folder, exist_ok=True)  # 確保輸出資料夾存在

# 獲取資料夾內的圖片名稱（按名稱排序）
image_files = sorted([f for f in os.listdir(input_folder) if f.endswith('.jpg') or f.endswith('.png')])

# 初始化結果列表
results = []

# 定義區域連通性分析的最小區域閾值
MIN_AREA_THRESHOLD = 500  # 可根據需要調整

# 遍歷相鄰圖片
for i in range(len(image_files) - 1):
    # 讀取相鄰兩張圖片
    img1_path = os.path.join(input_folder, image_files[i])
    img2_path = os.path.join(input_folder, image_files[i + 1])
    
    image1 = cv2.imread(img1_path, cv2.IMREAD_GRAYSCALE)
    image2 = cv2.imread(img2_path, cv2.IMREAD_GRAYSCALE)

    # --- 1. 圖像配準 ---
    _, binary1 = cv2.threshold(image1, 127, 255, cv2.THRESH_BINARY)
    _, binary2 = cv2.threshold(image2, 127, 255, cv2.THRESH_BINARY)

    warp_matrix = np.eye(2, 3, dtype=np.float32)
    criteria = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 5000, 1e-10)

    try:
        # 添加 inputMask 和 gaussFiltSize
        gaussFiltSize = 5  # 調整高斯濾波的大小（根據需要設定）
        cc, warp_matrix = cv2.findTransformECC(binary1, binary2, warp_matrix, cv2.MOTION_AFFINE, criteria, None, gaussFiltSize)
        binary2_aligned = cv2.warpAffine(binary2, warp_matrix, (binary1.shape[1], binary1.shape[0]), flags=cv2.INTER_LINEAR)
    except cv2.error as e:
        print(f"Warning: Image alignment failed for {image_files[i]} and {image_files[i+1]} due to {e}")
        continue

    # --- 2. 計算面積 ---
    area1 = np.sum(binary1 == 255)
    area2_aligned = np.sum(binary2_aligned == 255)
    growth_area = np.sum((binary2_aligned == 255) & (binary1 == 0))
    decomposition_area = np.sum((binary1 == 255) & (binary2_aligned == 0))

    # --- 3. 區域連通性分析 (Connected Component Analysis) ---
    def apply_cca(binary_img, min_area):
        num_labels, labels, stats, _ = cv2.connectedComponentsWithStats(binary_img, connectivity=8)
        filtered_binary = np.zeros_like(binary_img)
        for label in range(1, num_labels):  # 跳過背景（label 0）
            if stats[label, cv2.CC_STAT_AREA] >= min_area:
                filtered_binary[labels == label] = 255
        return filtered_binary

    growth_binary = ((binary2_aligned == 255) & (binary1 == 0)).astype(np.uint8) * 255
    decomposition_binary = ((binary1 == 255) & (binary2_aligned == 0)).astype(np.uint8) * 255

    growth_filtered = apply_cca(growth_binary, MIN_AREA_THRESHOLD)
    decomposition_filtered = apply_cca(decomposition_binary, MIN_AREA_THRESHOLD)

    # 計算過濾後的面積
    growth_area_filtered = np.sum(growth_filtered == 255)
    decomposition_area_filtered = np.sum(decomposition_filtered == 255)

    # --- 4. 視覺化生長與分解 ---
    visual_result = np.zeros((binary1.shape[0], binary1.shape[1], 3), dtype=np.uint8)
    visual_result[:, :, 1] = growth_filtered  # 生長用綠色
    visual_result[:, :, 2] = decomposition_filtered  # 分解用紅色

    visual_output_path = os.path.join(output_visual_folder, f"visual_{i+1}.jpg")
    cv2.imwrite(visual_output_path, visual_result)

    # --- 5. 儲存結果 ---
    results.append({
        'Image1': image_files[i],
        'Image2': image_files[i + 1],
        'Area1': area1,
        'Area2_Aligned': area2_aligned,
        'Growth_Area': growth_area,
        'Decomposition_Area': decomposition_area,
        'Growth_Area_Filtered': growth_area_filtered,
        'Decomposition_Area_Filtered': decomposition_area_filtered,
        'Visual_Result': visual_output_path
    })

# 將結果儲存為 CSV 檔案
df = pd.DataFrame(results)
df.to_csv(output_csv, index=False, encoding='utf-8-sig')

print(f"處理完成，結果已儲存到 {output_csv}")


In [ ]:
#align_images_ECC.py
#再次修正版本使輸出不亂跑
#會先仿射變換 (平移、旋轉、切變與縮放)
#失敗之後再用不切變與縮放，只有平移與旋轉（歐幾里得變換）

import cv2
import numpy as np
import os

def align_images(reference_image, target_image):
    """
    對齊影像，先嘗試平移、旋轉、切變與縮放。
    如果對齊失敗，則使用簡化的平移與旋轉模式。
    :param reference_image: 參考影像
    :param target_image: 目標影像
    :return: 對齊後的目標影像
    """
    a4_width, a4_height = 5100, 7021  # A4 尺寸 (600 DPI)

    # 調整輸入影像至 A4 大小
    reference_image = cv2.resize(reference_image, (a4_width, a4_height))
    target_image = cv2.resize(target_image, (a4_width, a4_height))

    # 轉換為灰階
    reference_gray = cv2.cvtColor(reference_image, cv2.COLOR_BGR2GRAY)
    target_gray = cv2.cvtColor(target_image, cv2.COLOR_BGR2GRAY)

    # 初始化仿射變換矩陣
    warp_matrix_affine = np.eye(2, 3, dtype=np.float32)
    warp_matrix_euclidean = np.eye(2, 3, dtype=np.float32)

    # 設定停止條件
    criteria = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 5000, 1e-10)

    # 優先嘗試仿射變換
    try:
        _, warp_matrix_affine = cv2.findTransformECC(
            reference_gray, target_gray, warp_matrix_affine, cv2.MOTION_AFFINE, 
            criteria, None, 5
        )
        aligned_image = cv2.warpAffine(
            target_image, warp_matrix_affine,
            (a4_width, a4_height),
            flags=cv2.INTER_LINEAR + cv2.WARP_INVERSE_MAP,
            borderMode=cv2.BORDER_CONSTANT,
            borderValue=(0, 0, 0)
        )
    except cv2.error:
        print("仿射變換對齊失敗，嘗試簡化的平移與旋轉模式")
        try:
            _, warp_matrix_euclidean = cv2.findTransformECC(
                reference_gray, target_gray, warp_matrix_euclidean, cv2.MOTION_EUCLIDEAN, 
                criteria, None, 5
            )
            aligned_image = cv2.warpAffine(
                target_image, warp_matrix_euclidean,
                (a4_width, a4_height),
                flags=cv2.INTER_LINEAR + cv2.WARP_INVERSE_MAP,
                borderMode=cv2.BORDER_CONSTANT,
                borderValue=(0, 0, 0)
            )
        except cv2.error:
            print("平移與旋轉對齊失敗，返回原始影像")
            aligned_image = target_image

    return aligned_image

def ensure_a4_size(image):
    """
    確保影像為A4大小，必要時進行填充或縮放。
    :param image: 輸入影像
    :return: A4大小的影像
    """
    a4_width, a4_height = 4960, 7014  # A4 尺寸 (600 DPI)
    
    # 創建A4畫布
    a4_canvas = np.full((a4_height, a4_width, 3), (0, 0, 0), dtype=np.uint8)
    
    if image is not None:
        h, w = image.shape[:2]
        scale = min(a4_width / w, a4_height / h)

        # 調整影像大小
        new_width = int(w * scale)
        new_height = int(h * scale)
        resized_image = cv2.resize(image, (new_width, new_height))

        # 計算居中位置
        y_offset = (a4_height - new_height) // 2
        x_offset = (a4_width - new_width) // 2

        # 放置影像於畫布上
        a4_canvas[y_offset:y_offset + new_height, x_offset:x_offset + new_width] = resized_image

    return a4_canvas

def process_images(folder_path, output_folder):
    """
    主函數：處理所有影像並確保輸出為A4大小。
    """
    os.makedirs(output_folder, exist_ok=True)

    image_files = sorted([f for f in os.listdir(folder_path) 
                         if f.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp'))])
    
    if not image_files:
        raise ValueError("資料夾中未找到影像檔案")

    reference_path = os.path.join(folder_path, image_files[0])
    reference_image = cv2.imread(reference_path)
    if reference_image is None:
        raise ValueError(f"無法讀取參考影像: {reference_path}")

    reference_image = ensure_a4_size(reference_image)
    output_path = os.path.join(output_folder, f"aligned_a4_1.png")
    cv2.imwrite(output_path, reference_image)
    print(f"已儲存參考影像: {output_path}")

    for i, image_file in enumerate(image_files[1:], 2):
        target_path = os.path.join(folder_path, image_file)
        target_image = cv2.imread(target_path)
        
        if target_image is None:
            print(f"警告：無法讀取影像 {target_path}，跳過處理")
            continue

        target_image = ensure_a4_size(target_image)
        aligned_image = align_images(reference_image, target_image)
        final_image = ensure_a4_size(aligned_image)
        
        output_path = os.path.join(output_folder, f"aligned_a4_{i}.png")
        cv2.imwrite(output_path, final_image)
        print(f"已儲存對齊影像: {output_path}")

if __name__ == "__main__":
    input_folder = r"E:\Shitephen\Output log for FU hinoki from ARATA\FU hinoki 1st take (1-565)\jpge files\20241116_052823_1b\postprocess\postprocess_test"
    output_folder = r"E:\Shitephen\Output log for FU hinoki from ARATA\FU hinoki 1st take (1-565)\jpge files\20241116_052823_1b\postprocess\postprocess_test\aligned_images"
    process_images(input_folder, output_folder)


In [ ]:
#ECC_fitting.py
# ECC (Enhanced Correlation Coefficient) fitting
# 目前最好
# 加上了區域連通性分析
# 篩噪點在計算生長和分解量之後
# 使平移+旋轉+切變+縮放 試圖對準各種形狀的根
# 如果對齊失敗，會用簡單的平移+旋轉

import cv2
import os
import numpy as np
import pandas as pd

# 資料夾路徑
input_folder = r"E:\Shitephen\Output log for FU hinoki from ARATA\FU hinoki 1st take (1-565)\jpge files\20241116_052823_1b\postprocess\postprocess_test\aligned_images"
output_csv = r"E:\Shitephen\Output log for FU hinoki from ARATA\FU hinoki 1st take (1-565)\jpge files\20241116_052823_1b\postprocess\postprocess_test\results_ECC_CCA.csv"
output_visual_folder = os.path.join(input_folder, "visual_results_ECC_CCA")

os.makedirs(output_visual_folder, exist_ok=True)  # 確保輸出資料夾存在

# 獲取資料夾內的圖片名稱（按名稱排序）
image_files = sorted([f for f in os.listdir(input_folder) if f.endswith('.jpg') or f.endswith('.png')])

# 初始化結果列表
results = []

# 定義區域連通性分析的最小區域閾值
MIN_AREA_THRESHOLD = 500  # 可根據需要調整

# 遍歷相鄰圖片
for i in range(len(image_files) - 1):
    # 讀取相鄰兩張圖片
    img1_path = os.path.join(input_folder, image_files[i])
    img2_path = os.path.join(input_folder, image_files[i + 1])
    
    image1 = cv2.imread(img1_path, cv2.IMREAD_GRAYSCALE)
    image2 = cv2.imread(img2_path, cv2.IMREAD_GRAYSCALE)

    # --- 1. 圖像配準 ---
    _, binary1 = cv2.threshold(image1, 127, 255, cv2.THRESH_BINARY)
    _, binary2 = cv2.threshold(image2, 127, 255, cv2.THRESH_BINARY)

    warp_matrix = np.eye(2, 3, dtype=np.float32)
    criteria = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 5000, 1e-10)

    try:
        # ECC 對齊
        cc, warp_matrix = cv2.findTransformECC(
            binary1, 
            binary2, 
            warp_matrix, 
            cv2.MOTION_AFFINE, 
            criteria, 
            None, 
            5  # gaussFiltSize
        )
        binary2_aligned = cv2.warpAffine(binary2, warp_matrix, (binary1.shape[1], binary1.shape[0]), flags=cv2.INTER_LINEAR)
    except cv2.error as e:
        print(f"ECC alignment failed for {image_files[i]} and {image_files[i+1]}, fallback to simple translation and rotation.")

        # 簡單平移和旋轉對齊
        warp_matrix = np.array([[1, 0, 0], [0, 1, 0]], dtype=np.float32)  # 初始化平移矩陣
        center = (binary1.shape[1] // 2, binary1.shape[0] // 2)  # 中心點
        rotation_matrix = cv2.getRotationMatrix2D(center, angle=0, scale=1)  # 旋轉矩陣（角度可調整）

        # 應用平移和旋轉
        binary2_aligned = cv2.warpAffine(binary2, rotation_matrix, (binary1.shape[1], binary1.shape[0]), flags=cv2.INTER_LINEAR)

    # --- 2. 計算面積 ---
    area1 = np.sum(binary1 == 255)
    area2_aligned = np.sum(binary2_aligned == 255)
    growth_area = np.sum((binary2_aligned == 255) & (binary1 == 0))
    decomposition_area = np.sum((binary1 == 255) & (binary2_aligned == 0))

    # --- 3. 區域連通性分析 (Connected Component Analysis) ---
    def apply_cca(binary_img, min_area):
        num_labels, labels, stats, _ = cv2.connectedComponentsWithStats(binary_img, connectivity=8)
        filtered_binary = np.zeros_like(binary_img)
        for label in range(1, num_labels):  # 跳過背景（label 0）
            if stats[label, cv2.CC_STAT_AREA] >= min_area:
                filtered_binary[labels == label] = 255
        return filtered_binary

    growth_binary = ((binary2_aligned == 255) & (binary1 == 0)).astype(np.uint8) * 255
    decomposition_binary = ((binary1 == 255) & (binary2_aligned == 0)).astype(np.uint8) * 255

    growth_filtered = apply_cca(growth_binary, MIN_AREA_THRESHOLD)
    decomposition_filtered = apply_cca(decomposition_binary, MIN_AREA_THRESHOLD)

    # 計算過濾後的面積
    growth_area_filtered = np.sum(growth_filtered == 255)
    decomposition_area_filtered = np.sum(decomposition_filtered == 255)

    # --- 4. 視覺化生長與分解 ---
    visual_result = np.zeros((binary1.shape[0], binary1.shape[1], 3), dtype=np.uint8)
    visual_result[:, :, 1] = growth_filtered  # 生長用綠色
    visual_result[:, :, 2] = decomposition_filtered  # 分解用紅色

    visual_output_path = os.path.join(output_visual_folder, f"visual_{i+1}.jpg")
    cv2.imwrite(visual_output_path, visual_result)

    # --- 5. 儲存結果 ---
    results.append({
        'Image1': image_files[i],
        'Image2': image_files[i + 1],
        'Area1': area1,
        'Area2_Aligned': area2_aligned,
        'Growth_Area': growth_area,
        'Decomposition_Area': decomposition_area,
        'Growth_Area_Filtered': growth_area_filtered,
        'Decomposition_Area_Filtered': decomposition_area_filtered,
        'Visual_Result': visual_output_path
    })

# 將結果儲存為 CSV 檔案
df = pd.DataFrame(results)
df.to_csv(output_csv, index=False, encoding='utf-8-sig')

print(f"處理完成，結果已儲存到 {output_csv}")


In [ ]:
#以上是目前穩定版

In [ ]:
#以下是融合UI

In [3]:
# 改良版 GUI 創建
import tkinter as tk
from tkinter import filedialog, messagebox, ttk
import os
import threading
import queue
import time


class Application(tk.Tk):
    def __init__(self):
        super().__init__()
        # 設定窗口標題為樹木符號 🌳
        self.title("🌳 Root turnover sensor 根のターンオーバーセンサー")
        
        self.geometry("600x350")
        self.configure(bg="#f7f7f7")
        self.iconbitmap("")  # 如果有圖標，可以設置圖標文件，如有需求

        self.folder_path = ""
        self.output_folder = ""
        #self.progress_var = tk.DoubleVar()
        #self.elapsed_time_var = tk.StringVar(value="Elapsed Time 経過時間: 0.00 s")
        
        #self.style = ttk.Style()
        #self.style.theme_use("clam")
        #self.style.configure("TButton", font=("Arial", 12), padding=5)
        #self.style.configure("TProgressbar", thickness=20)

        self.queue = queue.Queue()  # Initialize the queue here
        self.create_widgets()

    def create_widgets(self):
        # 標題標籤
        title_label = tk.Label(
            self,
            text="Turnover Sensor",
            font=("Helvetica", 18, "bold"),
            fg="#333",
            bg="#f7f7f7",
        )
        title_label.pack(pady=10)

        # 選擇資料夾按鈕
        self.select_button = ttk.Button(
            self, text="📁 Select Folder フォルダを選択", command=self.select_folder
        )
        self.select_button.pack(pady=10)

        # 對齊按鈕
        self.align_button = ttk.Button(
            self, text="🖼️ Align Images 画像の整列", command=self.start_align_thread
        )
        self.align_button.pack(pady=10)

        # 擬合按鈕
        self.fitting_button = ttk.Button(
            self, text="📊 ECC Fitting 擬合分析", command=self.start_fitting_thread
        )
        self.fitting_button.pack(pady=10)

        # 進度條
        #self.progress_bar = ttk.Progressbar(self, variable=self.progress_var, maximum=100)
        #self.progress_bar.pack(pady=15, fill="x", padx=20)

        # 經過時間標籤
        #self.time_label = tk.Label(
        #    self,
        #    textvariable=self.elapsed_time_var,
        #    font=("Arial", 12),
        #    fg="#555",
        #    bg="#f7f7f7",
        #)
        #self.time_label.pack(pady=10)

    def select_folder(self):
        self.folder_path = filedialog.askdirectory(title="Select Folder")
        if self.folder_path:
            self.output_folder = os.path.join(self.folder_path, "aligned_images")
            os.makedirs(self.output_folder, exist_ok=True)

    def start_align_thread(self):
        if not self.folder_path:
            messagebox.showerror("Error", "Please select a folder.")
            return
        threading.Thread(target=self.start_align_process, daemon=True).start()

    def start_fitting_thread(self):
        if not self.folder_path:
            messagebox.showerror("Error", "Please select a folder.")
            return
        threading.Thread(target=self.start_fitting_process, daemon=True).start()

    def start_align_process(self):
        start_time = time.time()
        try:
            process_images(self.folder_path, self.output_folder, self.queue_progress)
            elapsed_time = time.time() - start_time
            self.queue.put(("done", elapsed_time))
        except Exception as e:
            self.queue.put(("error", str(e)))

    def start_fitting_process(self):
        start_time = time.time()
        try:
            output_csv = os.path.join(self.folder_path, "ECC_results.csv")
            visual_folder = os.path.join(self.folder_path, "ECC_visuals")
            process_ecc_fitting(self.output_folder, output_csv, visual_folder, self.queue_progress)
            elapsed_time = time.time() - start_time
            self.queue.put(("done", elapsed_time))
        except Exception as e:
            self.queue.put(("error", str(e)))

    def queue_progress(self, progress):
        self.queue.put(("progress", progress))

    def process_queue(self):
        try:
            while True:
                item = self.queue.get_nowait()
                if item[0] == "progress":
                    self.progress_var.set(item[1])
                elif item[0] == "done":
                    elapsed_time = item[1]
                    self.elapsed_time_var.set(f"Elapsed Time 経過時間: {elapsed_time:.2f} s")
                    messagebox.showinfo("Success", "Processing complete!")
                elif item[0] == "error":
                    messagebox.showerror("Error", item[1])
        except queue.Empty:
            pass
        self.after(100, self.process_queue)


if __name__ == "__main__":
    app = Application()
    app.mainloop()


In [4]:
#三合一版
# 加上了區域連通性分析
# 篩噪點在計算生長和分解量之後
# 使平移+旋轉+切變+縮放 試圖對準各種形狀的根
# 如果對齊失敗，會用簡單的平移+旋轉

import os
import cv2
import numpy as np
import pandas as pd
import tkinter as tk
from tkinter import filedialog, messagebox, ttk
import threading
import queue
import time

def align_images(reference_image, target_image):
    """
    對齊影像，先嘗試平移、旋轉、切變與縮放。
    如果對齊失敗，則使用簡化的平移與旋轉模式。
    """
    a4_width, a4_height = 5100, 7021  # A4 尺寸 (600 DPI)

    # 調整輸入影像至 A4 大小
    reference_image = cv2.resize(reference_image, (a4_width, a4_height))
    target_image = cv2.resize(target_image, (a4_width, a4_height))

    # 轉換為灰階
    reference_gray = cv2.cvtColor(reference_image, cv2.COLOR_BGR2GRAY)
    target_gray = cv2.cvtColor(target_image, cv2.COLOR_BGR2GRAY)

    # 初始化仿射變換矩陣
    warp_matrix_affine = np.eye(2, 3, dtype=np.float32)
    warp_matrix_euclidean = np.eye(2, 3, dtype=np.float32)

    # 設定停止條件
    criteria = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 5000, 1e-10)

    # 優先嘗試仿射變換
    try:
        _, warp_matrix_affine = cv2.findTransformECC(
            reference_gray, target_gray, warp_matrix_affine, cv2.MOTION_AFFINE, 
            criteria, None, 5
        )
        aligned_image = cv2.warpAffine(
            target_image, warp_matrix_affine,
            (a4_width, a4_height),
            flags=cv2.INTER_LINEAR + cv2.WARP_INVERSE_MAP,
            borderMode=cv2.BORDER_CONSTANT,
            borderValue=(0, 0, 0)
        )
    except cv2.error:
        print("仿射變換對齊失敗，嘗試簡化的平移與旋轉模式")
        try:
            _, warp_matrix_euclidean = cv2.findTransformECC(
                reference_gray, target_gray, warp_matrix_euclidean, cv2.MOTION_EUCLIDEAN, 
                criteria, None, 5
            )
            aligned_image = cv2.warpAffine(
                target_image, warp_matrix_euclidean,
                (a4_width, a4_height),
                flags=cv2.INTER_LINEAR + cv2.WARP_INVERSE_MAP,
                borderMode=cv2.BORDER_CONSTANT,
                borderValue=(0, 0, 0)
            )
        except cv2.error:
            print("平移與旋轉對齊失敗，返回原始影像")
            aligned_image = target_image

    return aligned_image

def ensure_a4_size(image):
    """
    確保影像為A4大小，必要時進行填充或縮放。
    """
    a4_width, a4_height = 4960, 7014  # A4 尺寸 (600 DPI)
    
    # 創建A4畫布
    a4_canvas = np.full((a4_height, a4_width, 3), (0, 0, 0), dtype=np.uint8)
    
    if image is not None:
        h, w = image.shape[:2]
        scale = min(a4_width / w, a4_height / h)

        # 調整影像大小
        new_width = int(w * scale)
        new_height = int(h * scale)
        resized_image = cv2.resize(image, (new_width, new_height))

        # 計算居中位置
        y_offset = (a4_height - new_height) // 2
        x_offset = (a4_width - new_width) // 2

        # 放置影像於畫布上
        a4_canvas[y_offset:y_offset + new_height, x_offset:x_offset + new_width] = resized_image

    return a4_canvas

def process_images(input_folder, output_folder, progress_callback=None):
    """
    處理所有影像並確保輸出為A4大小。
    """
    os.makedirs(output_folder, exist_ok=True)

    image_files = sorted([f for f in os.listdir(input_folder) 
                         if f.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp'))])
    
    if not image_files:
        raise ValueError("資料夾中未找到影像檔案")

    for i, image_file in enumerate(image_files, 1):
        target_path = os.path.join(input_folder, image_file)
        target_image = cv2.imread(target_path)
        
        if target_image is None:
            print(f"警告：無法讀取影像 {target_path}，跳過處理")
            continue

        # 第一張圖直接儲存，其餘圖片與第一張對齊
        if i == 1:
            reference_image = ensure_a4_size(target_image)
        else:
            target_image = ensure_a4_size(target_image)
            target_image = align_images(reference_image, target_image)
        
        output_path = os.path.join(output_folder, f"aligned_a4_{i}.png")
        cv2.imwrite(output_path, target_image)
        print(f"已儲存對齊影像: {output_path}")
        
        # 更新進度回呼
        if progress_callback:
            progress_callback((i / len(image_files)) * 100)

def process_ecc_fitting(input_folder, output_csv, output_visual_folder, progress_callback=None):
    """
    執行ECC擬合分析。
    """
    os.makedirs(output_visual_folder, exist_ok=True)

    # 獲取資料夾內的圖片名稱（按名稱排序）
    image_files = sorted([f for f in os.listdir(input_folder) if f.endswith('.png') or f.endswith('.jpg')])

    # 初始化結果列表
    results = []

    # 定義區域連通性分析的最小區域閾值
    MIN_AREA_THRESHOLD = 500  # 可根據需要調整

    # 遍歷相鄰圖片
    for i in range(len(image_files) - 1):
        # 讀取相鄰兩張圖片
        img1_path = os.path.join(input_folder, image_files[i])
        img2_path = os.path.join(input_folder, image_files[i + 1])
        
        image1 = cv2.imread(img1_path, cv2.IMREAD_GRAYSCALE)
        image2 = cv2.imread(img2_path, cv2.IMREAD_GRAYSCALE)

        # --- 1. 圖像配準 ---
        _, binary1 = cv2.threshold(image1, 127, 255, cv2.THRESH_BINARY)
        _, binary2 = cv2.threshold(image2, 127, 255, cv2.THRESH_BINARY)

        warp_matrix = np.eye(2, 3, dtype=np.float32)
        criteria = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 5000, 1e-10)

        try:
            # ECC 對齊
            cc, warp_matrix = cv2.findTransformECC(
                binary1, 
                binary2, 
                warp_matrix, 
                cv2.MOTION_AFFINE, 
                criteria, 
                None, 
                5  # gaussFiltSize
            )
            binary2_aligned = cv2.warpAffine(binary2, warp_matrix, (binary1.shape[1], binary1.shape[0]), flags=cv2.INTER_LINEAR)
        except cv2.error as e:
            print(f"ECC alignment failed for {image_files[i]} and {image_files[i+1]}, fallback to simple translation and rotation.")
            warp_matrix = np.array([[1, 0, 0], [0, 1, 0]], dtype=np.float32)
            center = (binary1.shape[1] // 2, binary1.shape[0] // 2)
            rotation_matrix = cv2.getRotationMatrix2D(center, angle=0, scale=1)
            binary2_aligned = cv2.warpAffine(binary2, rotation_matrix, (binary1.shape[1], binary1.shape[0]), flags=cv2.INTER_LINEAR)

        # 計算面積和分析
        area1 = np.sum(binary1 == 255)
        area2_aligned = np.sum(binary2_aligned == 255)
        growth_area = np.sum((binary2_aligned == 255) & (binary1 == 0))
        decomposition_area = np.sum((binary1 == 255) & (binary2_aligned == 0))

        # 區域連通性分析
        def apply_cca(binary_img, min_area):
            num_labels, labels, stats, _ = cv2.connectedComponentsWithStats(binary_img, connectivity=8)
            filtered_binary = np.zeros_like(binary_img)
            for label in range(1, num_labels):
                if stats[label, cv2.CC_STAT_AREA] >= min_area:
                    filtered_binary[labels == label] = 255
            return filtered_binary

        growth_binary = ((binary2_aligned == 255) & (binary1 == 0)).astype(np.uint8) * 255
        decomposition_binary = ((binary1 == 255) & (binary2_aligned == 0)).astype(np.uint8) * 255

        growth_filtered = apply_cca(growth_binary, MIN_AREA_THRESHOLD)
        decomposition_filtered = apply_cca(decomposition_binary, MIN_AREA_THRESHOLD)

        growth_area_filtered = np.sum(growth_filtered == 255)
        decomposition_area_filtered = np.sum(decomposition_filtered == 255)

        # 視覺化
        visual_result = np.zeros((binary1.shape[0], binary1.shape[1], 3), dtype=np.uint8)
        visual_result[:, :, 1] = growth_filtered
        visual_result[:, :, 2] = decomposition_filtered

        visual_output_path = os.path.join(output_visual_folder, f"visual_{i+1}.jpg")
        cv2.imwrite(visual_output_path, visual_result)

        results.append({
            'Image1': image_files[i],
            'Image2': image_files[i + 1],
            'Area1': area1,
            'Area2_Aligned': area2_aligned,
            'Growth_Area': growth_area,
            'Decomposition_Area': decomposition_area,
            'Growth_Area_Filtered': growth_area_filtered,
            'Decomposition_Area_Filtered': decomposition_area_filtered,
            'Visual_Result': visual_output_path
        })
        
        # 更新進度回呼
        if progress_callback:
            progress_callback(((i + 1) / (len(image_files) - 1)) * 100)

    # 將結果儲存為 CSV 檔案
    df = pd.DataFrame(results)
    df.to_csv(output_csv, index=False, encoding='utf-8-sig')

class Application(tk.Tk):
    def __init__(self):
        super().__init__()
        self.title("🌳 Root Turnover Sensor")
        
        self.geometry("600x350")
        self.configure(bg="#f7f7f7")
        
        self.folder_path = ""
        self.output_folder = ""
        self.progress_var = tk.DoubleVar()
        #self.elapsed_time_var = tk.StringVar(value="Elapsed Time: 0.00 s")

        # 自己輸入圖片的pixels
        self.width_var = tk.StringVar(value="4960")  # 600 DPI
        self.height_var = tk.StringVar(value="7014")  # 600 DPI

        self.queue = queue.Queue()
        self.create_widgets()
        self.after(100, self.process_queue)
        
        # Add queue processing method to the main event loop
        self.after(100, self.process_queue)

    def create_widgets(self):
        title_label = tk.Label(
            self,
            text="Turnover Sensor",
            font=("Helvetica", 18, "bold"),
            fg="#333",
            bg="#f7f7f7",
        )
        title_label.pack(pady=10)

        # Dimensions frame
        dim_frame = tk.Frame(self, bg="#f7f7f7")
        dim_frame.pack(pady=10)

        tk.Label(dim_frame, text="Width:", bg="#f7f7f7").grid(row=0, column=0, padx=5)
        width_entry = tk.Entry(dim_frame, textvariable=self.width_var, width=10)
        width_entry.grid(row=0, column=1, padx=5)

        tk.Label(dim_frame, text="Height:", bg="#f7f7f7").grid(row=0, column=2, padx=5)
        height_entry = tk.Entry(dim_frame, textvariable=self.height_var, width=10)
        height_entry.grid(row=0, column=3, padx=5)
        
        # Buttons
        self.select_button = ttk.Button(
            self, text="📁 Select Folder フォルダを選択", command=self.select_folder
        )
        self.select_button.pack(pady=10)

        self.align_button = ttk.Button(
            self, text="🖼️ Align Images 画像の整列", command=self.start_align_thread
        )
        self.align_button.pack(pady=10)

        self.fitting_button = ttk.Button(
            self, text="📊 ECC Fitting 擬合分析", command=self.start_fitting_thread
        )
        self.fitting_button.pack(pady=10)

        # Progress bar
        self.progress_bar = ttk.Progressbar(self, variable=self.progress_var, maximum=100)
        self.progress_bar.pack(pady=15, fill="x", padx=20)

        self.time_label = tk.Label(
            self,
            textvariable=self.elapsed_time_var,
            font=("Arial", 12),
            fg="#555",
            bg="#f7f7f7",
        )
        self.time_label.pack(pady=10)

    def select_folder(self):
        self.folder_path = filedialog.askdirectory(title="Select Folder")
        if self.folder_path:
            self.output_folder = os.path.join(self.folder_path, "aligned_images")
            os.makedirs(self.output_folder, exist_ok=True)

    def start_align_thread(self):
        if not self.folder_path:
            messagebox.showerror("Error", "Please select a folder.")
            return
        threading.Thread(target=self.start_align_process, daemon=True).start()

    def start_align_process(self):
        start_time = time.time()
        try:
            width = int(self.width_var.get())
            height = int(self.height_var.get())
            process_images(self.folder_path, self.output_folder, self.queue_progress)
            elapsed_time = time.time() - start_time
            self.queue.put(("done", elapsed_time))
        except Exception as e:
            self.queue.put(("error", str(e)))
            
    def start_fitting_thread(self):
        if not self.folder_path or not os.listdir(self.aligned_folder):
            messagebox.showerror("Error", "Please align images first.")
            return
        threading.Thread(target=self.start_fitting_process, daemon=True).start()

    def start_fitting_process(self):
        start_time = time.time()
        try:
            width = int(self.width_var.get())
            height = int(self.height_var.get())
            output_csv = os.path.join(self.folder_path, "ECC_results.csv")
            visual_folder = os.path.join(self.folder_path, "ECC_visuals")
            process_ecc_fitting(self.output_folder, output_csv, visual_folder, self.queue_progress)
            elapsed_time = time.time() - start_time
            self.queue.put(("done", elapsed_time))
        except Exception as e:
            self.queue.put(("error", str(e)))

    def queue_progress(self, progress):
        self.queue.put(("progress", progress))

    def process_queue(self):
        try:
            while True:
                item = self.queue.get_nowait()
                if item[0] == "progress":
                    self.progress_var.set(item[1])
                elif item[0] == "done":
                    elapsed_time = item[1]
                    self.elapsed_time_var.set(f"Elapsed Time 経過時間: {elapsed_time:.2f} s")
                    messagebox.showinfo("Success", "Processing complete!")
                elif item[0] == "error":
                    messagebox.showerror("Error", item[1])
                    return
        except queue.Empty:
            pass
        self.after(100, self.process_queue)

if __name__ == "__main__":
    app = Application()
    app.mainloop()

已儲存對齊影像: E:/Shitephen/Output log for FU hinoki from ARATA/FU hinoki 1st take (1-565)/jpge files/20241116_052823_1b/postprocess/postprocess_test\aligned_images\aligned_a4_1.png
已儲存對齊影像: E:/Shitephen/Output log for FU hinoki from ARATA/FU hinoki 1st take (1-565)/jpge files/20241116_052823_1b/postprocess/postprocess_test\aligned_images\aligned_a4_2.png
已儲存對齊影像: E:/Shitephen/Output log for FU hinoki from ARATA/FU hinoki 1st take (1-565)/jpge files/20241116_052823_1b/postprocess/postprocess_test\aligned_images\aligned_a4_3.png


In [ ]:
#三合一版
# 加上了區域連通性分析
# 篩噪點在計算生長和分解量之後
# 使平移+旋轉+切變+縮放 試圖對準各種形狀的根
# 如果對齊失敗，會用簡單的平移+旋轉
# 在UI加上自定義的長和寬
# 把fitting的路徑改的嚴謹一點

import os
import cv2
import numpy as np
import pandas as pd
import tkinter as tk
from tkinter import filedialog, messagebox, ttk
import threading
import queue
import time

def align_images(reference_image, target_image):
    """
    對齊影像，先嘗試平移、旋轉、切變與縮放。
    如果對齊失敗，則使用簡化的平移與旋轉模式。
    a4_width, a4_height = 5100, 7021  # e.g. A4 尺寸 (600 DPI)
    """

    # 調整輸入影像至 A4 大小
    reference_image = cv2.resize(reference_image, (a4_width, a4_height))
    target_image = cv2.resize(target_image, (a4_width, a4_height))

    # 轉換為灰階
    reference_gray = cv2.cvtColor(reference_image, cv2.COLOR_BGR2GRAY)
    target_gray = cv2.cvtColor(target_image, cv2.COLOR_BGR2GRAY)

    # 初始化仿射變換矩陣
    warp_matrix_affine = np.eye(2, 3, dtype=np.float32)
    warp_matrix_euclidean = np.eye(2, 3, dtype=np.float32)

    # 設定停止條件
    criteria = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 5000, 1e-10)

    # 優先嘗試仿射變換
    try:
        _, warp_matrix_affine = cv2.findTransformECC(
            reference_gray, target_gray, warp_matrix_affine, cv2.MOTION_AFFINE, 
            criteria, None, 5
        )
        aligned_image = cv2.warpAffine(
            target_image, warp_matrix_affine,
            (a4_width, a4_height),
            flags=cv2.INTER_LINEAR + cv2.WARP_INVERSE_MAP,
            borderMode=cv2.BORDER_CONSTANT,
            borderValue=(0, 0, 0)
        )
    except cv2.error:
        print("仿射變換對齊失敗，嘗試簡化的平移與旋轉模式")
        try:
            _, warp_matrix_euclidean = cv2.findTransformECC(
                reference_gray, target_gray, warp_matrix_euclidean, cv2.MOTION_EUCLIDEAN, 
                criteria, None, 5
            )
            aligned_image = cv2.warpAffine(
                target_image, warp_matrix_euclidean,
                (a4_width, a4_height),
                flags=cv2.INTER_LINEAR + cv2.WARP_INVERSE_MAP,
                borderMode=cv2.BORDER_CONSTANT,
                borderValue=(0, 0, 0)
            )
        except cv2.error:
            print("平移與旋轉對齊失敗，返回原始影像")
            aligned_image = target_image

    return aligned_image

def ensure_a4_size(image):
    """
    確保影像為A4大小，必要時進行填充或縮放。
    a4_width, a4_height = 5100, 7021  # e.g. A4 尺寸 (600 DPI)    
    """
    
    # 創建A4畫布
    a4_canvas = np.full((a4_height, a4_width, 3), (0, 0, 0), dtype=np.uint8)
    
    if image is not None:
        h, w = image.shape[:2]
        scale = min(a4_width / w, a4_height / h)

        # 調整影像大小
        new_width = int(w * scale)
        new_height = int(h * scale)
        resized_image = cv2.resize(image, (new_width, new_height))

        # 計算居中位置
        y_offset = (a4_height - new_height) // 2
        x_offset = (a4_width - new_width) // 2

        # 放置影像於畫布上
        a4_canvas[y_offset:y_offset + new_height, x_offset:x_offset + new_width] = resized_image

    return a4_canvas

def process_images(input_folder, output_folder, progress_callback=None):
    """
    處理所有影像並確保輸出為A4大小。
    """
    os.makedirs(output_folder, exist_ok=True)

    image_files = sorted([f for f in os.listdir(input_folder) 
                         if f.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp'))])
    
    if not image_files:
        raise ValueError("資料夾中未找到影像檔案")

    for i, image_file in enumerate(image_files, 1):
        target_path = os.path.join(input_folder, image_file)
        target_image = cv2.imread(target_path)
        
        if target_image is None:
            print(f"警告：無法讀取影像 {target_path}，跳過處理")
            continue

        # 第一張圖直接儲存，其餘圖片與第一張對齊
        if i == 1:
            reference_image = ensure_a4_size(target_image)
        else:
            target_image = ensure_a4_size(target_image)
            target_image = align_images(reference_image, target_image)
        
        output_path = os.path.join(output_folder, f"aligned_a4_{i}.png")
        cv2.imwrite(output_path, target_image)
        print(f"已儲存對齊影像: {output_path}")
        
        # 更新進度回呼
        if progress_callback:
            progress_callback((i / len(image_files)) * 100)

def process_ecc_fitting(input_folder, output_csv, output_visual_folder, progress_callback=None):
    """
    執行ECC擬合分析。
    """
    os.makedirs(output_visual_folder, exist_ok=True)

    # 獲取資料夾內的圖片名稱（按名稱排序）
    image_files = sorted([f for f in os.listdir(input_folder) if f.endswith('.png') or f.endswith('.jpg')])

    # 初始化結果列表
    results = []

    # 定義區域連通性分析的最小區域閾值
    MIN_AREA_THRESHOLD = 500  # 可根據需要調整

    # 遍歷相鄰圖片
    for i in range(len(image_files) - 1):
        # 讀取相鄰兩張圖片
        img1_path = os.path.join(input_folder, image_files[i])
        img2_path = os.path.join(input_folder, image_files[i + 1])
        
        image1 = cv2.imread(img1_path, cv2.IMREAD_GRAYSCALE)
        image2 = cv2.imread(img2_path, cv2.IMREAD_GRAYSCALE)

        # --- 1. 圖像配準 ---
        _, binary1 = cv2.threshold(image1, 127, 255, cv2.THRESH_BINARY)
        _, binary2 = cv2.threshold(image2, 127, 255, cv2.THRESH_BINARY)

        warp_matrix = np.eye(2, 3, dtype=np.float32)
        criteria = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 5000, 1e-10)

        try:
            # ECC 對齊
            cc, warp_matrix = cv2.findTransformECC(
                binary1, 
                binary2, 
                warp_matrix, 
                cv2.MOTION_AFFINE, 
                criteria, 
                None, 
                5  # gaussFiltSize
            )
            binary2_aligned = cv2.warpAffine(binary2, warp_matrix, (binary1.shape[1], binary1.shape[0]), flags=cv2.INTER_LINEAR)
        except cv2.error as e:
            print(f"ECC alignment failed for {image_files[i]} and {image_files[i+1]}, fallback to simple translation and rotation.")
            warp_matrix = np.array([[1, 0, 0], [0, 1, 0]], dtype=np.float32)
            center = (binary1.shape[1] // 2, binary1.shape[0] // 2)
            rotation_matrix = cv2.getRotationMatrix2D(center, angle=0, scale=1)
            binary2_aligned = cv2.warpAffine(binary2, rotation_matrix, (binary1.shape[1], binary1.shape[0]), flags=cv2.INTER_LINEAR)

        # 計算面積和分析
        area1 = np.sum(binary1 == 255)
        area2_aligned = np.sum(binary2_aligned == 255)
        growth_area = np.sum((binary2_aligned == 255) & (binary1 == 0))
        decomposition_area = np.sum((binary1 == 255) & (binary2_aligned == 0))

        # 區域連通性分析
        def apply_cca(binary_img, min_area):
            num_labels, labels, stats, _ = cv2.connectedComponentsWithStats(binary_img, connectivity=8)
            filtered_binary = np.zeros_like(binary_img)
            for label in range(1, num_labels):
                if stats[label, cv2.CC_STAT_AREA] >= min_area:
                    filtered_binary[labels == label] = 255
            return filtered_binary

        growth_binary = ((binary2_aligned == 255) & (binary1 == 0)).astype(np.uint8) * 255
        decomposition_binary = ((binary1 == 255) & (binary2_aligned == 0)).astype(np.uint8) * 255

        growth_filtered = apply_cca(growth_binary, MIN_AREA_THRESHOLD)
        decomposition_filtered = apply_cca(decomposition_binary, MIN_AREA_THRESHOLD)

        growth_area_filtered = np.sum(growth_filtered == 255)
        decomposition_area_filtered = np.sum(decomposition_filtered == 255)

        # 視覺化
        visual_result = np.zeros((binary1.shape[0], binary1.shape[1], 3), dtype=np.uint8)
        visual_result[:, :, 1] = growth_filtered
        visual_result[:, :, 2] = decomposition_filtered

        visual_output_path = os.path.join(output_visual_folder, f"visual_{i+1}.jpg")
        cv2.imwrite(visual_output_path, visual_result)

        results.append({
            'Image1': image_files[i],
            'Image2': image_files[i + 1],
            'Area1': area1,
            'Area2_Aligned': area2_aligned,
            'Growth_Area': growth_area,
            'Decomposition_Area': decomposition_area,
            'Growth_Area_Filtered': growth_area_filtered,
            'Decomposition_Area_Filtered': decomposition_area_filtered,
            'Visual_Result': visual_output_path
        })
        
        # 更新進度回呼
        if progress_callback:
            progress_callback(((i + 1) / (len(image_files) - 1)) * 100)

    # 將結果儲存為 CSV 檔案
    df = pd.DataFrame(results)
    df.to_csv(output_csv, index=False, encoding='utf-8-sig')

class Application(tk.Tk):
    def __init__(self):
        super().__init__()
        self.title("🌳 Root Turnover Sensor")
        
        self.geometry("600x350")
        self.configure(bg="#f7f7f7")
        
        self.folder_path = ""
        self.output_folder = ""
        self.progress_var = tk.DoubleVar()
        #self.elapsed_time_var = tk.StringVar(value="Elapsed Time: 0.00 s")

        # 自己輸入圖片的pixels
        self.width_var = tk.StringVar(value="4960")  # 600 DPI
        self.height_var = tk.StringVar(value="7014")  # 600 DPI

        self.queue = queue.Queue()
        self.create_widgets()
        self.after(100, self.process_queue)
        
        # Add queue processing method to the main event loop
        self.after(100, self.process_queue)

    def create_widgets(self):
        title_label = tk.Label(
            self,
            text="Turnover Sensor",
            font=("Helvetica", 18, "bold"),
            fg="#333",
            bg="#f7f7f7",
        )
        title_label.pack(pady=10)

        # Dimensions frame
        dim_frame = tk.Frame(self, bg="#f7f7f7")
        dim_frame.pack(pady=10)

        tk.Label(dim_frame, text="Width:", bg="#f7f7f7").grid(row=0, column=0, padx=5)
        width_entry = tk.Entry(dim_frame, textvariable=self.width_var, width=10)
        width_entry.grid(row=0, column=1, padx=5)

        tk.Label(dim_frame, text="Height:", bg="#f7f7f7").grid(row=0, column=2, padx=5)
        height_entry = tk.Entry(dim_frame, textvariable=self.height_var, width=10)
        height_entry.grid(row=0, column=3, padx=5)
        
        # Buttons
        self.select_button = ttk.Button(
            self, text="📁 Select Folder フォルダを選択", command=self.select_folder
        )
        self.select_button.pack(pady=10)

        self.align_button = ttk.Button(
            self, text="🖼️ Align Images 画像の整列", command=self.start_align_thread
        )
        self.align_button.pack(pady=10)

        self.fitting_button = ttk.Button(
            self, text="📊 ECC Fitting 擬合分析", command=self.start_fitting_thread
        )
        self.fitting_button.pack(pady=10)

        # Progress bar
        self.progress_bar = ttk.Progressbar(self, variable=self.progress_var, maximum=100)
        self.progress_bar.pack(pady=15, fill="x", padx=20)

        self.time_label = tk.Label(
            self,
            textvariable=self.elapsed_time_var,
            font=("Arial", 12),
            fg="#555",
            bg="#f7f7f7",
        )
        self.time_label.pack(pady=10)

    def select_folder(self):
        self.folder_path = filedialog.askdirectory(title="Select Folder")
        if self.folder_path:
            self.output_folder = os.path.join(self.folder_path, "aligned_images")
            os.makedirs(self.output_folder, exist_ok=True)

    def start_align_thread(self):
        if not self.folder_path:
            messagebox.showerror("Error", "Please select a folder.")
            return
        threading.Thread(target=self.start_align_process, daemon=True).start()

    def start_align_process(self):
        start_time = time.time()
        try:
            width = int(self.width_var.get())
            height = int(self.height_var.get())
            process_images(
                self.folder_path, 
                self.output_folder,
                width, 
                height, 
                self.queue_progress
            )
            elapsed_time = time.time() - start_time
            self.queue.put(("done", elapsed_time))
        except Exception as e:
            self.queue.put(("error", str(e)))
            
    def start_fitting_thread(self):
        if not self.folder_path or not os.listdir(self.aligned_folder):
            messagebox.showerror("Error", "Please select the postprocess folder from ARATA or align images first.")
            return
        threading.Thread(target=self.start_fitting_process, daemon=True).start()

    def start_fitting_process(self):
        start_time = time.time()
        try:
            width = int(self.width_var.get())
            height = int(self.height_var.get())
            output_csv = os.path.join(self.folder_path, "ECC_results.csv")
            visual_folder = os.path.join(self.folder_path, "ECC_visuals")
            process_ecc_fitting(
                self.output_folder, 
                output_csv, 
                visual_folder, 
                width,
                height,
                self.queue_progress
            )
            elapsed_time = time.time() - start_time
            self.queue.put(("done", elapsed_time))
        except Exception as e:
            self.queue.put(("error", str(e)))

    def queue_progress(self, progress):
        self.queue.put(("progress", progress))

    def process_queue(self):
        try:
            while True:
                item = self.queue.get_nowait()
                if item[0] == "progress":
                    self.progress_var.set(item[1])
                elif item[0] == "done":
                    elapsed_time = item[1]
                    self.elapsed_time_var.set(f"Elapsed Time 経過時間: {elapsed_time:.2f} s")
                    messagebox.showinfo("Success", "Processing complete!")
                elif item[0] == "error":
                    messagebox.showerror("Error", item[1])
                    return
        except queue.Empty:
            pass
        self.after(100, self.process_queue)

if __name__ == "__main__":
    app = Application()
    app.mainloop()